In [ ]:
spark

In [ ]:
sc

In [ ]:
%%configure -f \
{"conf": {"spark.jars.packages": "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.2"}}

In [ ]:
!pip install spark-nlp

In [ ]:
!pip install azureml.fsspec

In [ ]:
!pip install wordcloud

In [ ]:
import pandas as pd
import numpy as np
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from azure.storage.blob import BlobServiceClient
import io
from azureml.fsspec import AzureMachineLearningFileSystem

from azure.storage.blob import BlobServiceClient
# from azureml.core import AzureMachineLearningFileSystem
import io
import pandas as pd
import os
import sys
from pyspark.sql import SparkSession
from pyspark import SparkContext 
from pyspark.sql.functions import col, lower, count, length, unix_timestamp, current_timestamp, to_date, desc
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType
import re
from datetime import datetime
import pandas as pd
from tabulate import tabulate
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
from pyspark.sql import functions as F
from scipy.stats import tstd
import nltk
from wordcloud import WordCloud,STOPWORDS
import matplotlib.pyplot as plt
# !pip install plotly
# !pip install wordcloud
import plotly.express as px

# download the nltk stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk_stopwords = set(stopwords.words('english'))

import pandas as pd
import numpy as np
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import lit
from pyspark.sql import Window
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from pyspark.ml.feature import Tokenizer, CountVectorizer, IDF, HashingTF, StopWordsRemover
import pyspark.sql.functions as F
from pyspark.sql.types import ArrayType, MapType, StringType
import string
from sparknlp.pretrained import PretrainedPipeline
import matplotlib.pyplot as plt

import nltk
nltk.download('stopwords')

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import SQLTransformer
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer, Normalizer, StopWordsCleaner, LemmatizerModel

In [ ]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

# Azure Storage Account details
storage_account_name = "group23astorage87e07d16d"
storage_account_access_key = "gkm2Ao5WwFH2W/6Udmx14mJyvglx8W74in3UXd8JrARi99bf7UFdFxnAJKnI5w4gk6ePiKTmH5XL+AStoA70xw=="
container_name = "azureml-blobstore-b5f84dac-6a3c-4f67-8b42-7075c37e11ae"

# Configure Spark to access Azure Blob Storage
conf = SparkConf()
conf.set("fs.azure.account.key." + storage_account_name + ".blob.core.windows.net", storage_account_access_key)

# Initialize Spark Session
spark = SparkSession.builder.appName("AzureBlobStorageRead").config(conf=conf).getOrCreate()

# Years to process
years = ['2021', '2022', '2023']

# Initialize an empty list to hold DataFrames
all_dfs = []

# Process each year
for year in years:
    # Specify the file path in the container for the current year
    file_path = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/cleaned/comments/yyyy={year}/*.parquet"

    # Read the parquet files into a DataFrame
    df = spark.read.format("parquet").load(file_path)

    # Append the DataFrame to the list
    all_dfs.append(df)

# Concatenate all DataFrames for all years
comments = all_dfs[0]
for df in all_dfs[1:]:
    comments = comments.union(df)

# Show the selected columns
comments.select("subreddit", "author", "body", "parent_id", "link_id", "id", "created_utc", "score").show()


In [ ]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

# Azure Storage Account details
storage_account_name = "group23astorage87e07d16d"
storage_account_access_key = "gkm2Ao5WwFH2W/6Udmx14mJyvglx8W74in3UXd8JrARi99bf7UFdFxnAJKnI5w4gk6ePiKTmH5XL+AStoA70xw=="
container_name = "azureml-blobstore-b5f84dac-6a3c-4f67-8b42-7075c37e11ae"

# Configure Spark to access Azure Blob Storage
conf = SparkConf()
conf.set("fs.azure.account.key." + storage_account_name + ".blob.core.windows.net", storage_account_access_key)

# Initialize Spark Session
spark = SparkSession.builder.appName("AzureBlobStorageRead").config(conf=conf).getOrCreate()

# Years to process
years = ['2021', '2022', '2023']

# Initialize an empty list to hold DataFrames
all_dfs = []

# Process each year
for year in years:
    # Specify the file path in the container for the current year
    file_path = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/cleaned/submissions/yyyy={year}/*.parquet"

    # Read the parquet files into a DataFrame
    df = spark.read.format("parquet").load(file_path)

    # Append the DataFrame to the list
    all_dfs.append(df)

# Concatenate all DataFrames for all years
submissions = all_dfs[0]
for df in all_dfs[1:]:
    submissions = submissions.union(df)

# Select specific columns
submissions.select("subreddit", "author", "title", "selftext", "created_utc", "num_comments", "score").show()


# Filter "AskWomen", "AskFeminists", "Feminism" by STEM Keywords

In [ ]:
from pyspark.sql.functions import col

# Subreddits to filter by keywords
keyword_subreddits = ["AskWomen", "AskFeminists", "Feminism"]
# Subreddits to include all comments from
include_all_subreddits = ["xxstem", "LadiesofScience", "womenEngineers"]

# Define keywords for case-insensitive search
keywords = ["STEM", "Science", "Technology", "Engineering", "Mathematics", "Process", "Design", "Model", "Plan", "Project"]
keywords_lower = [kw.lower() for kw in keywords]

# Filter the DataFrame
comments = comments.filter(
    (col("subreddit").isin(keyword_subreddits) & col("body").rlike('|'.join(keywords_lower))) |
    (col("subreddit").isin(include_all_subreddits))
)

# Show the filtered data
comments.select("subreddit", "author", "body", "parent_id", "link_id", "id", "created_utc", "score").show()

In [ ]:
from pyspark.sql.functions import col

# Subreddits to filter by keywords
keyword_subreddits = ["AskWomen", "AskFeminists", "Feminism"]
# Subreddits to include all submissions from
include_all_subreddits = ["xxstem", "LadiesofScience", "womenEngineers"]

# Define keywords for case-insensitive search
keywords = ["STEM", "science", "technology", "engineering", "mathematics", "process", "design", "model", "plan", "project"]
# Create a regex pattern to match any keyword (case-insensitive)
pattern = '|'.join([f"(?i){kw}" for kw in keywords])

# Filter the DataFrame
# Include all submissions from certain subreddits or those that match the keyword pattern in their title or selftext
submissions = submissions.filter(
    (col("subreddit").isin(keyword_subreddits) & (col("title").rlike(pattern) | col("selftext").rlike(pattern))) |
    col("subreddit").isin(include_all_subreddits)
)

# Show the filtered data
submissions.select("subreddit", "author", "title", "selftext", "created_utc", "num_comments", "score").show()

# Cleaning the Data

In [ ]:
# Remove rows with missing values in 'author' and 'body'
comments = comments.filter(col("author").isNotNull() & col("body").isNotNull())

# Assume that 'created_utc' should be a timestamp within the last 3 years
three_years_ago = unix_timestamp(current_timestamp()) - (3 * 365 * 24 * 60 * 60)
comments = comments.filter(
    unix_timestamp(col("created_utc")) > three_years_ago
)

# Show the filtered data
comments.select("subreddit", "author", "body", "parent_id", "link_id", "id", "created_utc", "score").show()

In [ ]:
# Remove rows with missing values in 'author' or 'title'
submissions = submissions.filter(col("author").isNotNull() & col("title").isNotNull())

# Assume that 'created_utc' should be a timestamp within the last 3 years
three_years_ago = unix_timestamp(current_timestamp()) - (3 * 365 * 24 * 60 * 60)
submissions = submissions.filter(
    unix_timestamp(col("created_utc")) > three_years_ago
)

# Show the filtered data
submissions.select("subreddit", "author", "title", "selftext", "created_utc", "num_comments",  "score").show()

# External Data

In [ ]:
from azure.storage.blob import BlobServiceClient
import pandas as pd

# Azure Storage Account details
storage_account_name = "group23astorage87e07d16d"
storage_account_key = "gkm2Ao5WwFH2W/6Udmx14mJyvglx8W74in3UXd8JrARi99bf7UFdFxnAJKnI5w4gk6ePiKTmH5XL+AStoA70xw=="
container_name = "azureml-blobstore-b5f84dac-6a3c-4f67-8b42-7075c37e11ae"
blob_name = "nsf23315-report-figures-tables-excels/nsf23315-fig001-001.xlsx"

# Create a blob client
blob_service_client = BlobServiceClient(account_url=f"https://{storage_account_name}.blob.core.windows.net", credential=storage_account_key)
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

# Download the blob to a local file
local_file_name = "downloaded_nsf23315_fig001_001.xlsx"
with open(local_file_name, "wb") as download_file:
    download_file.write(blob_client.download_blob().readall())

# Read the Excel file using pandas
df = pd.read_excel(local_file_name)

# Display the first few rows of the DataFrame
print(df.head())


In [ ]:
from pyspark.sql.functions import lit
from pyspark.sql import SparkSession
import pandas as pd

df_spark = spark.createDataFrame(df)

df_spark = df_spark.withColumn("dummy_key", lit(1))
comments = comments.withColumn("dummy_key", lit(1))

merged_df = comments.join(df_spark, "dummy_key")

merged_df = merged_df.drop("dummy_key")


In [ ]:
from pyspark.sql import functions as F

# Summary by Subreddit
summary_subreddit = merged_df.groupBy("subreddit").agg(
    F.count("id").alias("comment_count"),
    F.avg("score").alias("average_score"),
    F.avg("Total population").alias("avg_total_population")
)

summary_subreddit.show()


# Preprocessing the data

# Most Frequent Words in the Comments and Submissions Data

In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF
from pyspark.sql.functions import udf, col, explode, array_union
from pyspark.sql.types import StringType
import re

# Define a UDF for cleaning text
def clean_text(text):
    text = text.lower()
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove non-alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text.strip()

clean_text_udf = udf(clean_text, StringType())

# Preprocess and tokenize text for both datasets
comments = comments.withColumn("cleaned_body", clean_text_udf("body"))
submissions = submissions.withColumn("cleaned_title", clean_text_udf("title"))
submissions = submissions.withColumn("cleaned_selftext", clean_text_udf("selftext"))

tokenizer = Tokenizer(inputCol="cleaned_body", outputCol="words")
comments = tokenizer.transform(comments)

tokenizer = Tokenizer(inputCol="cleaned_title", outputCol="title_words")
submissions = tokenizer.transform(submissions)
tokenizer = Tokenizer(inputCol="cleaned_selftext", outputCol="selftext_words")
submissions = tokenizer.transform(submissions)

# Define additional stop words
additional_stop_words = ["like", "dont", "im", "one", "removed", "get", "also", "even", "really", "sa", "despite", "certainly"]


# Remove stop words
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
remover.setStopWords(remover.getStopWords() + additional_stop_words)
comments = remover.transform(comments)

remover = StopWordsRemover(inputCol="title_words", outputCol="filtered_title_words")
remover.setStopWords(remover.getStopWords() + additional_stop_words)
submissions = remover.transform(submissions)

remover = StopWordsRemover(inputCol="selftext_words", outputCol="filtered_selftext_words")
remover.setStopWords(remover.getStopWords() + additional_stop_words)
submissions = remover.transform(submissions)

# Combine title and selftext words for submissions
submissions = submissions.withColumn("combined_words", array_union("filtered_title_words", "filtered_selftext_words"))

## Finding Important Keywords and creating dummies

In [ ]:
# Education and Color
edu_keywords = ['education', 'school', 'university']
col_keywords = ['"hispanic"', 'black', 'asian', 'ethnicity']

# Regular expressions for the topics
edu_regex = '|'.join([re.escape(word) for word in edu_keywords])
col_regex = '|'.join([re.escape(word) for word in col_keywords])

# Define UDFs to check for the presence of topic keywords
def contains_tech(text):
    return int(bool(re.search(edu_regex, text.lower())))

def contains_sports(text):
    return int(bool(re.search(col_regex, text.lower())))

contains_edu_udf = udf(contains_tech, IntegerType())
contains_col_udf = udf(contains_sports, IntegerType())

# Apply UDFs to create dummy variables
comments = comments.withColumn("is_edu", contains_edu_udf("cleaned_body"))
comments = comments.withColumn("is_col", contains_col_udf("cleaned_body"))

# Calculating Word Frequencies

In [ ]:
# Calculate Word Frequencies
comments_exploded = comments.withColumn("word", explode("filtered_words"))
comments_exploded = comments_exploded.filter(col("word").rlike(r'\S'))

submissions_exploded = submissions.withColumn("word", explode("combined_words"))
submissions_exploded = submissions_exploded.filter(col("word").rlike(r'\S'))

comments_word_freq = comments_exploded.groupBy("word").count().sort("count", ascending=False)
submissions_word_freq = submissions_exploded.groupBy("word").count().sort("count", ascending=False)

# Display the tables
comments_word_freq.show(10)
submissions_word_freq.show(10)

In [ ]:
!pip install pandas tabulate pillow


In [ ]:
import pandas as pd
from tabulate import tabulate
from PIL import Image, ImageDraw, ImageFont

# Convert Spark DataFrame to Pandas DataFrame
comments_word_freq_pd = comments_word_freq.toPandas()
submissions_word_freq_pd = submissions_word_freq.toPandas()

# Function to convert DataFrame to image
def df_to_image(df, filename):
    # Convert DataFrame to table string
    table_str = tabulate(df.head(10), headers='keys', tablefmt='plain')

    # Create an image with white background
    img = Image.new('RGB', (1000, 300), color = (255, 255, 255))
    d = ImageDraw.Draw(img)

    # Use a truetype font (you might need to adjust the path to the font file)
    font = ImageFont.load_default()

    # Draw the table on the image
    d.multiline_text((10,10), table_str, fill=(0,0,0), font=font)

    # Save the image
    img.save(filename)

# Convert DataFrames to images
df_to_image(comments_word_freq_pd, 'comments_word_freq.png') # change path
df_to_image(submissions_word_freq_pd, 'submissions_word_freq.png') # change path


# Word Clouds after cleaning

In [ ]:
# Convert Spark DataFrame to Pandas DataFrame
comments_pd = comments_word_freq.toPandas()
submissions_pd = submissions_word_freq.toPandas()

# Create a dictionary of word frequencies for word cloud
comments_dict = dict(zip(comments_pd['word'], comments_pd['count']))
submissions_dict = dict(zip(submissions_pd['word'], submissions_pd['count']))

# Generate word cloud for comments
wordcloud_comments = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                min_font_size = 10).generate_from_frequencies(comments_dict)

# Plot and save the WordCloud image for comments                       
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud_comments) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
plt.savefig('wordcloud_comments.png')  # CHANGE PATH
plt.show()  # Close the plot to avoid displaying it in the notebook

# Generate word cloud for submissions
wordcloud_submissions = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                min_font_size = 10).generate_from_frequencies(submissions_dict)

# Plot and save the WordCloud image for submissions                       
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud_submissions) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
plt.savefig('wordcloud_submissions.png')  
plt.show()  

# Average distribution of text lengths for Submissions and Comments

In [ ]:
submissions.createOrReplaceTempView("submissions")
comments.createOrReplaceTempView("comments")

avg_subm_length_by_month = spark.sql("""
    SELECT
        subreddit,
        date_format(created_utc, "yyyy-MM") AS month,
        AVG(LENGTH(selftext)) AS avg_subm_length
    FROM submissions
    GROUP BY subreddit, month
    ORDER BY month
""")

avg_comm_length_by_month = spark.sql("""
    SELECT
        subreddit,
        date_format(created_utc, "yyyy-MM") AS month,
        AVG(LENGTH(body)) AS avg_comm_length
    FROM comments
    GROUP BY subreddit, month
    ORDER BY month
""")

# Convert the Spark DataFrames to Pandas DataFrames
avg_subm_length_by_month = avg_subm_length_by_month.toPandas()
avg_comm_length_by_month = avg_comm_length_by_month.toPandas()

# Create a line chart of average submission length by month
fig = px.line(
    avg_subm_length_by_month,
    x="month",
    y="avg_subm_length",
    color="subreddit",
    title="Average Submission Length by Month",
    labels={"avg_subm_length": "Average Submission Length"}
)

# Save the figure as an HTML file
fig.write_html("average_text_length_over_time_sub.html")


# Display the charts
fig.show()


In [ ]:
# Create a line chart of average comment length by month
fig = px.line(
    avg_comm_length_by_month,
    x="month",
    y="avg_comm_length",
    color="subreddit",
    title="Average Comment Length by Month",
    labels={"avg_comm_length": "Average Comments Length"}
)

# Save the figure as an HTML file
fig.write_html("average_text_length_over_time_com.html")

fig.show()

In [ ]:
!pwd

# Count Vectorizer

In [ ]:
from pyspark.ml.feature import CountVectorizer, IDF
from pyspark.sql.functions import udf, explode, col, max
from pyspark.sql.types import ArrayType, DoubleType

# Count Vectorizer and IDF for comments
cv_comments = CountVectorizer(inputCol="filtered_words", outputCol="raw_features")
cv_model_comments = cv_comments.fit(comments)
comments_featurized = cv_model_comments.transform(comments)

idf_comments = IDF(inputCol="raw_features", outputCol="features")
idf_model_comments = idf_comments.fit(comments_featurized)
comments_tfidf = idf_model_comments.transform(comments_featurized)

In [ ]:
comments_tfidf.select("filtered_words", "raw_features", "features").show(5)

In [ ]:
from pyspark.ml.feature import CountVectorizer, IDF
from pyspark.sql.functions import udf, explode
from pyspark.sql.types import ArrayType, DoubleType

# Count Vectorizer and IDF for submissions
cv_submissions = CountVectorizer(inputCol="combined_words", outputCol="raw_features")
cv_model_submissions = cv_submissions.fit(submissions)
submissions_featurized = cv_model_submissions.transform(submissions)

idf_submissions = IDF(inputCol="raw_features", outputCol="features")
idf_model_submissions = idf_submissions.fit(submissions_featurized)
submissions_tfidf = idf_model_submissions.transform(submissions_featurized)

In [ ]:
submissions_tfidf.select("combined_words", "raw_features", "features").show()

# TF-IDF SCORE CALCULATION FOR SUBMISSIONS AND COMMENTS DATASET

In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF
from pyspark.sql.functions import udf, col, explode, array_union, monotonically_increasing_id
from pyspark.sql.types import StringType, ArrayType, DoubleType
import re

# UDF to extract TF-IDF values from SparseVector - submissions and comments
def extract_tf_idf_values(features):
    return features.toArray().tolist()

extract_tf_idf_values_udf = udf(extract_tf_idf_values, ArrayType(DoubleType()))

In [ ]:
# Apply UDF to create a new column with TF-IDF values
comments_tfidf = comments_tfidf.withColumn("tf_idf_values", extract_tf_idf_values_udf("features"))

In [ ]:
comments_tfidf.select("filtered_words", "raw_features", "features", "tf_idf_values").show(5)

In [ ]:
submissions_tfidf.printSchema()

In [ ]:
comments_tfidf.printSchema()

In [ ]:
comments_exploded_tfidf = comments_tfidf.select("filtered_words", "tf_idf_values").withColumn("words", explode("filtered_words")).withColumn("tf_idf", explode("tf_idf_values"))


In [ ]:
comments_exploded_tfidf.select("filtered_words", "tf_idf_values", "words", "tf_idf").show(5)

In [ ]:
# Filter out words that are URLs
# Explode the words along with their indices
comments_exploded = comments_with_index.select(
    posexplode("indexed_words").alias("idx", "word_with_index")
)


In [ ]:
# Extract the word and its corresponding TF-IDF value
comments_exploded_tfidf = comments_exploded.select(
    col("word_with_index.word").alias("word"),
    col("tf_idf_values")[col("idx")].alias("tf_idf")
)

In [ ]:
# Group by word and find the maximum TF-IDF value for each word
top_comments_tf_idf = filtered_comments_tfidf.groupBy("word").agg(max("tf_idf").alias("max_tf_idf")).orderBy(col("max_tf_idf").desc())

In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF
from pyspark.sql.functions import udf, col, explode, array_union, monotonically_increasing_id, row_number
from pyspark.sql.window import Window
from pyspark.sql.types import StringType, ArrayType, DoubleType
import re
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF
from pyspark.sql.functions import udf, col, explode, array_union, max
from pyspark.sql.types import StringType, ArrayType, DoubleType
import re

submissions_tfidf = submissions_tfidf.withColumn("tf_idf_values", extract_tf_idf_values_udf("features"))



In [ ]:
submissions_tfidf.select("combined_words", "raw_features", "features", "tf_idf_values").show(5)

In [ ]:
submissions_exploded_tfidf = submissions_tfidf.select("combined_words", "tf_idf_values").withColumn("word", explode("combined_words")).withColumn("tf_idf", explode("tf_idf_values"))
submissions_exploded_tfidf = submissions_exploded_tfidf.filter(~col("word").rlike("^(http|https)"))



In [ ]:
submissions_exploded_tfidf.select("combined_words", "tf_idf_values").show(5)

In [ ]:
# Group by word and find the maximum TF-IDF value for each word
top_submissions_tf_idf = submissions_exploded_tfidf.groupBy("word").agg(max("tf_idf").alias("max_tf_idf")).orderBy(col("max_tf_idf").desc())


In [ ]:
# Show top results
top_comments_tf_idf.show(10)


In [ ]:
# Display the results
top_submissions_tf_idf.show(10)

# Sentiment Analysis for Comments and Submissions Datasets 

# Twitter Model

In [ ]:
from pyspark.ml import Pipeline
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import UniversalSentenceEncoder, SentimentDLModel
from sparknlp.pretrained import PretrainedPipeline
import sparknlp

In [ ]:
# Assuming 'comments' and 'submissions' are your DataFrames and they have a column 'text' for analysis
comments = comments.withColumnRenamed("body", "text")
# submissions = submissions.withColumnRenamed("title", "text")

# Define the pipeline
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en") \
    .setInputCols(["document"]) \
    .setOutputCol("sentence_embeddings")

sentimentdl = SentimentDLModel.pretrained(name="sentimentdl_use_twitter", lang="en") \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("sentiment")

sentiment_pipeline1 = Pipeline(stages=[documentAssembler, use, sentimentdl])

# Fit the pipeline to your data
model1 = sentiment_pipeline1.fit(comments)

# Transform the data
result1 = model1.transform(comments)

# Show some results
# result.select("text", "sentiment.result").show(truncate=False)

In [ ]:
# Concatenate 'title' and 'selftext' into a single text column if needed
submissions = submissions.withColumn("text", F.concat_ws(" ", submissions.title, submissions.selftext))

# Define the pipeline (same as before)
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en") \
    .setInputCols(["document"]) \
    .setOutputCol("sentence_embeddings")

sentimentdl = SentimentDLModel.pretrained(name="sentimentdl_use_twitter", lang="en") \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("sentiment")

sentiment_pipeline2 = Pipeline(stages=[documentAssembler, use, sentimentdl])

# Fit the pipeline to your data
model2 = sentiment_pipeline2.fit(submissions)

# Transform the data
result2 = model2.transform(submissions)

# Show some results
# result.select("text", "sentiment.result").show(truncate=False)

# Vivek Sentiment Model

In [ ]:
from pyspark.ml import Pipeline
from sparknlp.base import DocumentAssembler, Finisher
from sparknlp.annotator import Tokenizer, Normalizer, ViveknSentimentModel
import sparknlp

# Define the pipeline
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("tokenized")

normalizer = Normalizer() \
    .setInputCols(["tokenized"]) \
    .setOutputCol("normalized")

vivekn = ViveknSentimentModel.pretrained() \
    .setInputCols(["document", "normalized"]) \
    .setOutputCol("vn_sentiment")

finisher = Finisher() \
    .setInputCols(["vn_sentiment"]) \
    .setOutputCols(["sentiment"])

sentiment_pipeline3 = Pipeline(stages=[documentAssembler, tokenizer, normalizer, vivekn, finisher])

# Fit the pipeline to your data
model3 = sentiment_pipeline3.fit(comments)

# Transform the data
result3 = model3.transform(comments)

# Show some results
# result.select("body", "sentiment").show(truncate=False)

In [ ]:
result3.select("text", "sentiment").show(10)

In [ ]:
# Prepare your data for the 'submissions' DataFrame
submissions = submissions.withColumn("text", F.concat_ws(" ", submissions.title, submissions.selftext))

# The pipeline remains the same, just change the input column for the DocumentAssembler
documentAssembler.setInputCol("text")

# Fit the pipeline to your data
model4 = sentiment_pipeline3.fit(submissions)

# Transform the data
result4 = model4.transform(submissions)

# Show some results
# result.select("text", "sentiment").show(truncate=False)


# IMDB Sentiment Model

In [ ]:
from pyspark.ml import Pipeline
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import UniversalSentenceEncoder, SentimentDLModel
import sparknlp

# Initialize Spark NLP
spark = sparknlp.start()

MODEL_3 = "sentimentdl_use_imdb"

# Define the pipeline
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en") \
    .setInputCols(["document"]) \
    .setOutputCol("sentence_embeddings")

sentimentDl = SentimentDLModel.pretrained(name=MODEL_3, lang='en') \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("sentiment")

sentiment_pipeline4 = Pipeline(stages=[documentAssembler, use, sentimentDl])

# Fit the pipeline to your data
model5 = sentiment_pipeline4.fit(comments)

# Transform the data
result5 = model5.transform(comments)

# Show some results
# result.select("body", "sentiment").show(truncate=False)


In [ ]:
# Prepare your data for the 'submissions' DataFrame
submissions = submissions.withColumn("text", F.concat_ws(" ", submissions.title, submissions.selftext))

# The pipeline remains the same, just change the input column for the DocumentAssembler
documentAssembler.setInputCol("text")

# Fit the pipeline to your data
model6 = sentiment_pipeline4.fit(submissions)

# Transform the data
result6 = model6.transform(submissions)

# Show some results
# result.select("text", "sentiment").show(truncate=False)


In [ ]:
# Explode the sentiment results for the 'comments' DataFrame
exploded_results1 = result1.withColumn("sentiment", explode("sentiment.result"))
exploded_results2 = result2.withColumn("sentiment", explode("sentiment.result"))
# exploded_results3 = result3.withColumn("sentiment", explode("sentiment.result"))
# exploded_results4 = result4.withColumn("sentiment", explode("sentiment.result"))
exploded_results5 = result5.withColumn("sentiment", explode("sentiment.result"))
exploded_results6 = result6.withColumn("sentiment", explode("sentiment.result"))

from pyspark.sql.functions import explode
sentiment_counts1 = exploded_results1.groupBy("sentiment").count()
sentiment_counts2 = exploded_results2.groupBy("sentiment").count()
# sentiment_counts3 = exploded_results3.groupBy("sentiment").count()
# sentiment_counts4 = exploded_results4.groupBy("sentiment").count()
sentiment_counts5 = exploded_results5.groupBy("sentiment").count()
sentiment_counts6 = exploded_results6.groupBy("sentiment").count()


In [ ]:
from pyspark.sql.functions import explode, col, lit
from pyspark.sql.window import Window
from pyspark.sql.functions import sum as _sum

# Function to calculate and show sentiment percentages
def show_sentiment_percentages(results):
    # Explode the sentiment results
    exploded_results = results.withColumn("sentiment", explode("sentiment.result"))

    # Count the total number of sentiments
    total_count = exploded_results.count()

    # Calculate the count of each sentiment type
    sentiment_counts = exploded_results.groupBy("sentiment").count()

    # Calculate the percentage of each sentiment type
    sentiment_percentages = sentiment_counts.withColumn("percentage", col("count") / lit(total_count) * 100)

    # Show the sentiment percentages
    sentiment_percentages.show()

# Apply the function to the results for 'comments'

show_sentiment_percentages(result1)
show_sentiment_percentages(result2)
# show_sentiment_percentages(result3)
# show_sentiment_percentages(result4)
show_sentiment_percentages(result5)
show_sentiment_percentages(result6)


In [ ]:
from pyspark.sql.functions import col, lit

# Function to calculate and show sentiment percentages
def calculate_sentiment_percentages(result):
    # Count the total number of sentiments
    total_count = result.count()

    # Calculate the count of each sentiment type
    sentiment_counts = result.groupBy("sentiment").count()

    # Calculate the percentage of each sentiment type
    sentiment_percentages = sentiment_counts.withColumn("percentage", col("count") / lit(total_count) * 100)

    return sentiment_percentages

# Calculate sentiment percentages for result3
sentiment_percentages3 = calculate_sentiment_percentages(result3)
sentiment_percentages3.show()

# Calculate sentiment percentages for result4
sentiment_percentages4 = calculate_sentiment_percentages(result4)
sentiment_percentages4.show()


Disclaimer: The work has been by us. However, we have taken assissstance from ChatGPT for code commenting and cleaning.